In [1]:
#dependencies
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine

import time

In [2]:
# read in data 
real_fake_df = pd.read_csv('fake_job_postings.csv', index_col=0)
real_fake_df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
job_id,,,,,,,,,,,,,,,,,
1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
# df.describe, all of the data
real_fake_df.describe()

,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000
mean,0.042897,0.795302,0.491723,0.048434
std,0.202631,0.403492,0.499945,0.214688
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [4]:
# count of the fraudulent (1) postings and the actual (0)
real_fake_df["fraudulent"].value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

In [5]:
# Type of data in each column
real_fake_df.dtypes

title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

In [6]:
# Dropping Columns 
# Will add a column for "has and does not have benefits"
# Feel free to adjust for the machine learning model - just add or take away column names in the "" inside of the []
real_fake_df=real_fake_df.drop(columns=["salary_range", "department", "function"], axis=1)
real_fake_df.dtypes

title                  object
location               object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
fraudulent              int64
dtype: object

In [7]:
# Showing the amount of unique inputs in each column, easiest to focus on the ones with the smallest. "required_education" and "required_experience"
# both had small amounts of unique inputs
real_fake_df.nunique()

title                  11231
location                3105
company_profile         1709
description            14801
requirements           11968
benefits                6205
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
industry                 131
fraudulent                 2
dtype: int64

In [8]:
# Count of NaN cells
real_fake_df.isna().count()

title                  17880
location               17880
company_profile        17880
description            17880
requirements           17880
benefits               17880
telecommuting          17880
has_company_logo       17880
has_questions          17880
employment_type        17880
required_experience    17880
required_education     17880
industry               17880
fraudulent             17880
dtype: int64

In [9]:
# Drop rows with NaN entries, creating new dataframe "cleaned_posting_df"
cleaned_posting_df=real_fake_df.dropna()

In [10]:
# Count of rows left after dropping NaN
len(cleaned_posting_df)

4676

In [11]:
# count of the fraudulent (1) postings and the actual (0) of the cleaned data 
cleaned_posting_df["fraudulent"].value_counts()

0    4514
1     162
Name: fraudulent, dtype: int64

In [12]:
# Displaying cleaned data 
cleaned_posting_df.head()

,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,fraudulent
job_id,,,,,,,,,,,,,,
4,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,0
5,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,0
7,Head of Content (m/f),"DE, BE, Berlin","Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,0
15,Account Executive - Sydney,"AU, NSW, Sydney",Adthena is the UK’s leading competitive intell...,Are you interested in a satisfying and financi...,You’ll need to be smart and passionate and hav...,"In return we'll pay you well, give you some ow...",0,1,0,Full-time,Associate,Bachelor's Degree,Internet,0
16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,0


In [13]:
# Determine the # of fraudulent posts based on employment type 
cleaned_posting_df.value_counts(['employment_type','fraudulent'])

employment_type  fraudulent
Full-time        0             3860
Contract         0              326
Part-time        0              191
Full-time        1              153
Temporary        0               78
Other            0               59
Contract         1                4
Part-time        1                4
Other            1                1
dtype: int64

In [14]:
# Creating a binary column to capture Full-time (1) and everything else (0) 
cleaned_posting_df.loc[(cleaned_posting_df['employment_type'] == 'Full-time'), 'full_time'] = 1
cleaned_posting_df.loc[(cleaned_posting_df['employment_type'] != "Full-time"), 'full_time'] = 0
 
cleaned_posting_df.head(5)    

/var/folders/_c/4fsnf3gs2x1bwj83jsfn90400000gn/T/ipykernel_6990/1941406935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_posting_df.loc[(cleaned_posting_df['employment_type'] == 'Full-time'), 'full_time'] = 1


,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,fraudulent,full_time
job_id,,,,,,,,,,,,,,,
4,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,0,1.0
5,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,0,1.0
7,Head of Content (m/f),"DE, BE, Berlin","Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,0,1.0
15,Account Executive - Sydney,"AU, NSW, Sydney",Adthena is the UK’s leading competitive intell...,Are you interested in a satisfying and financi...,You’ll need to be smart and passionate and hav...,"In return we'll pay you well, give you some ow...",0,1,0,Full-time,Associate,Bachelor's Degree,Internet,0,1.0
16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,0,1.0


In [15]:
# Determine the # of fraudulent posts based on required experience
cleaned_posting_df.value_counts(['required_experience','fraudulent'])

required_experience  fraudulent
Mid-Senior level     0             1504
Associate            0             1183
Entry level          0             1083
Not Applicable       0              374
Director             0              167
Internship           0              136
Executive            0               67
Mid-Senior level     1               62
Entry level          1               55
Associate            1               30
Director             1               12
Not Applicable       1                3
dtype: int64

In [16]:
# Creating a binary column to capture Mid-Senior, Entry level, Associate, and Director-levl (1) and everything else (0) 
cleaned_posting_df.loc[(cleaned_posting_df['required_experience'] == 'Mid-Senior level') | (cleaned_posting_df['required_experience'] == 'Entry level') | (cleaned_posting_df['required_experience'] == 'Associate') | (cleaned_posting_df['required_experience'] == 'Director'),'experience'] = 1
cleaned_posting_df.loc[(cleaned_posting_df['required_experience'] != 'Mid-Senior level') & (cleaned_posting_df['required_experience'] != 'Entry level') & (cleaned_posting_df['required_experience'] != 'Associate') & (cleaned_posting_df['required_experience'] != 'Director'),'experience'] = 0

cleaned_posting_df.head(5) 

/var/folders/_c/4fsnf3gs2x1bwj83jsfn90400000gn/T/ipykernel_6990/3108255098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_posting_df.loc[(cleaned_posting_df['required_experience'] == 'Mid-Senior level') | (cleaned_posting_df['required_experience'] == 'Entry level') | (cleaned_posting_df['required_experience'] == 'Associate') | (cleaned_posting_df['required_experience'] == 'Director'),'experience_binary'] = 1


,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,fraudulent,full_time,experience_binary
job_id,,,,,,,,,,,,,,,,
4,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,0,1.0,1.0
5,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,0,1.0,1.0
7,Head of Content (m/f),"DE, BE, Berlin","Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,0,1.0,1.0
15,Account Executive - Sydney,"AU, NSW, Sydney",Adthena is the UK’s leading competitive intell...,Are you interested in a satisfying and financi...,You’ll need to be smart and passionate and hav...,"In return we'll pay you well, give you some ow...",0,1,0,Full-time,Associate,Bachelor's Degree,Internet,0,1.0,1.0
16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,0,1.0,0.0


In [17]:
# Determine the # of fraudulent posts based on required education
cleaned_posting_df.value_counts(['required_education','fraudulent'])

required_education                 fraudulent
Bachelor's Degree                  0             2556
Unspecified                        0              724
High School or equivalent          0              701
Master's Degree                    0              173
Associate Degree                   0              166
High School or equivalent          1               86
Certification                      0               62
Some College Coursework Completed  0               56
Bachelor's Degree                  1               43
Professional                       0               33
Vocational                         0               23
Doctorate                          0               15
Master's Degree                    1               13
Certification                      1               11
Associate Degree                   1                5
Unspecified                        1                3
Vocational - HS Diploma            0                3
Some High School Coursework        0

In [26]:
#High School or Certification
cleaned_posting_df.loc[(cleaned_posting_df['required_experience'] == 'High School or equivalent') | (cleaned_posting_df['required_experience'] == 'Certification'),'education_level'] = 1

#Associate, Bacherlor’s, Master’s or Doctorate Degree
cleaned_posting_df.loc[(cleaned_posting_df['required_experience'] == 'Associate Degree') | (cleaned_posting_df['required_education'] == "Bachelor's Degree") | (cleaned_posting_df['required_education'] == "Master's Degree") | (cleaned_posting_df['required_education'] == 'Doctorate'),'education_level'] = 2

# Everything else
cleaned_posting_df.loc[(cleaned_posting_df['required_experience'] != 'Associate Degree') & (cleaned_posting_df['required_education'] != "Bachelor's Degree") & (cleaned_posting_df['required_education'] != "Master's Degree") & (cleaned_posting_df['required_education'] != 'Doctorate') & (cleaned_posting_df['required_experience'] != 'High School or equivalent') & (cleaned_posting_df['required_experience'] != 'Certification'), 'education_level'] = 0


cleaned_posting_df.head()


,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,fraudulent,full_time,experience_binary,education_level
job_id,,,,,,,,,,,,,,,,,
4,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,0,1.0,1.0,2.0
5,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,0,1.0,1.0,2.0
7,Head of Content (m/f),"DE, BE, Berlin","Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,0,1.0,1.0,2.0
15,Account Executive - Sydney,"AU, NSW, Sydney",Adthena is the UK’s leading competitive intell...,Are you interested in a satisfying and financi...,You’ll need to be smart and passionate and hav...,"In return we'll pay you well, give you some ow...",0,1,0,Full-time,Associate,Bachelor's Degree,Internet,0,1.0,1.0,2.0
16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,0,1.0,0.0,2.0


In [30]:
#Dropping Duplicate Columns
updated_df=cleaned_posting_df.drop(columns=["employment_type", "required_experience", "required_education"], axis=1)
updated_df.head()

,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,industry,fraudulent,full_time,experience_binary,education_level
job_id,,,,,,,,,,,,,,
4,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Computer Software,0,1.0,1.0,2.0
5,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Hospital & Health Care,0,1.0,1.0,2.0
7,Head of Content (m/f),"DE, BE, Berlin","Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Online Media,0,1.0,1.0,2.0
15,Account Executive - Sydney,"AU, NSW, Sydney",Adthena is the UK’s leading competitive intell...,Are you interested in a satisfying and financi...,You’ll need to be smart and passionate and hav...,"In return we'll pay you well, give you some ow...",0,1,0,Internet,0,1.0,1.0,2.0
16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Facilities Services,0,1.0,0.0,2.0


In [32]:
updated_df.to_csv('updated_postings_code.csv')